Задачи

Импортируйте библиотеку pandas как pd. Загрузите два датасета user_data и logs. Проверьте размер таблицы, типы переменных, наличие пропущенных значений, описательную статистику.
-Какой клиент совершил больше всего успешных операций? (success == True)
-С какой платформы осуществляется наибольшее количество успешных операций?
-Какую платформу предпочитают премиальные клиенты?
-Визуализируйте распределение возраста клиентов в зависимости от типа клиента (премиум или нет)
-Постройте график распределения числа успешных операций
-Визуализируйте число успешных операций, сделанных на платформе computer, в зависимости от возраста, используя sns.countplot (x – возраст, y – число успешных операций). Клиенты какого возраста совершили наибольшее количество успешных действий?

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plp

In [2]:
#Загрузите два датасета user_data и logs
user_data = pd.read_csv('user_data.csv')
#user_data = 'C:/Users/avk06/КарповСтепик/Минипроект2_Урок3/user_data.csv' - сначала создаем путь до файла (~/ потом Tab)
#pd.read_csv(user_data) - затем считываем
logs = pd.read_csv('logs.csv')

1. Проверьте размер таблицы, типы переменных, наличие пропущенных значений, описательную статистику.

In [53]:
#Сколько уникальных значений принимает переменная platform
logs['platform'].nunique()

3

In [13]:
user_data.shape

(2954, 3)

In [15]:
logs.shape

(4500, 4)

In [14]:
user_data.dtypes

client     int64
premium     bool
age        int64
dtype: object

In [16]:
logs.dtypes

client       int64
success       bool
platform    object
time         int64
dtype: object

In [30]:
#наличие пропущенных значений
user_data.isnull().sum()

client     0
premium    0
age        0
dtype: int64

In [31]:
#наличие пропущенных значений
logs.isnull().sum()

client      0
success     0
platform    0
time        0
dtype: int64

In [19]:
#описательная статистика
user_data.describe()

,client,age
count,2954.000000,2954.000000
mean,50767.886594,44.196005
std,28800.505847,14.248336
min,40.000000,20.000000
25%,25639.500000,32.000000
50%,51569.500000,44.000000
75%,75963.250000,56.000000
max,99973.000000,69.000000


In [21]:
logs.describe()

,client,time
count,4500.000000,4.500000e+03
mean,50998.758000,1.585440e+09
std,28374.472495,2.314866e+04
min,40.000000,1.585400e+09
25%,27056.500000,1.585420e+09
50%,52659.000000,1.585440e+09
75%,76001.250000,1.585461e+09
max,99993.000000,1.585480e+09


2. Какой клиент совершил больше всего успешных операций? (success == True)

In [3]:
logs

,client,success,platform,time
0,40177,True,phone,1585412212
1,61468,True,phone,1585425658
2,35604,False,phone,1585459894
3,9961,True,phone,1585400189
4,10931,True,computer,1585468629
...,...,...,...,...
4495,64496,False,computer,1585459072
4496,73299,True,phone,1585458370
4497,8967,True,phone,1585465121
4498,52880,False,phone,1585455362


In [4]:
logs_count = logs\
    .query('success==True')\
    .groupby('client', as_index = False).count()\
    .sort_values('success', ascending = False)\
    .rename(columns={'success':'success_count'})

In [5]:
logs_count.head()

,client,success_count,platform,time
332,12179,41,41,41
1758,61468,41,41,41
1487,52870,41,41,41
1759,61473,41,41,41
2626,92584,41,41,41


In [6]:
success_max = logs_count.query('success_count==41').sort_values('client')

In [7]:
success_max

,client,success_count,platform,time
332,12179,41,41,41
810,28719,41,41,41
1020,36165,41,41,41
1487,52870,41,41,41
1758,61468,41,41,41
1759,61473,41,41,41
2231,78349,41,41,41
2345,82563,41,41,41
2626,92584,41,41,41


In [8]:
#используя медот to_list() серию из client переведем в список
client_max = success_max.client.to_list()
client_max

[12179, 28719, 36165, 52870, 61468, 61473, 78349, 82563, 92584]

In [9]:
#способ преобразования списка client_max в строку (неэффективный): заитерируем каждый элемент списка и запринтем его 
#с разделителем запятая
for client in client_max:
    print(client, end=', ')

12179, 28719, 36165, 52870, 61468, 61473, 78349, 82563, 92584, 

In [10]:
#более лучший способ
', '.join([str(client) for client in client_max])

'12179, 28719, 36165, 52870, 61468, 61473, 78349, 82563, 92584'

3. С какой платформы было совершено наибольшее количество успешных операций?

In [11]:
logs.head()

,client,success,platform,time
0,40177,True,phone,1585412212
1,61468,True,phone,1585425658
2,35604,False,phone,1585459894
3,9961,True,phone,1585400189
4,10931,True,computer,1585468629


In [12]:
(
logs.query('success == True')
    .groupby('platform', as_index = False)
    .agg({'client' : 'count'})
    .sort_values('client', ascending = False)
)

,platform,client
1,phone,2019
0,computer,1090
2,tablet,180


In [16]:
# или
logs.query('success == True').platform.value_counts().idxmax()

'phone'

4. Какая платформа наиболее популярна среди премиумных клиентов?

5. Визуализируйте распределение возраста клиентов в зависимости от типа клиента (премиум или нет), используя библиотеку seaborn с параметрами цвета по умолчанию. Какой график получится в результате?

6. Постройте график распределения числа успешных операций: 

x – число успешных операций
y – число клиентов, совершивших такое количество успешных операций

Выберите подходящие утверждения:

7. Визуализируйте число успешных операций, сделанных на платформе computer, в зависимости от возраста, используя sns.countplot. По оси x – возраст, по оси y – число успешных операций. Если размер графика получается небольшим, можно добавить plt.figure(figsize=(12, 8)) в начало соответствующей ячейки. 

Полезные ссылки:

sns.countplot – документация к типу графика
figsize – про размер графиков
Вопрос: Клиенты какого возраста совершили наибольшее количество успешных действий?